# Parallel Chains

Similarly to how sequential chains connect chains in series, parallel chains connect chains in parallel. This is a useful abstraction when you want to independently process the same input with multiple different chains, possibly asynchronously.

In the following examples, we wlll show:
- how we can use `ParallelChain` to take a list of chains and apply each independently to the same input.
- how we can nest `ParallelChain`s inside `ParallelChain`s

In [1]:
import pprint
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ParallelChain

## Simple Example

In [2]:
llm = OpenAI(temperature=0.9)

input_variables=['product']

prompt_1 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good mission statement for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

prompt_3 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good slogan for a company that makes {product}?",
)
chain_3 = LLMChain(llm=llm, prompt=prompt_3)

prompt_4 = PromptTemplate(
    input_variables=input_variables,
    template="What are some core values for a company that makes {product}?",
)
chain_4 = LLMChain(llm=llm, prompt=prompt_4)

When the `concurrent` flag is set to `True`, we can run the child chains concurrently. The `concurrent` flag is set to `True` by default.

In [3]:
parallel_chain = ParallelChain(
    input_variables=input_variables,
    chains={
        'name': chain_1, 
        'mission': chain_2, 
        'slogan': chain_3,
        'values': chain_4
    },
    verbose=True,
    concurrent=True
)
s = time.perf_counter()
output = parallel_chain("colorful socks")
pprint.pprint(output)
print('\033[1m' + f"Concurrent executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="mission" started.
Child chain for key="slogan" started.
Child chain for key="values" started.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


Child chain for key="slogan" finished after 5.33 seconds.
Child chain for key="name" finished after 5.36 seconds.
Child chain for key="mission" finished after 6.80 seconds.
Child chain for key="values" finished after 8.01 seconds.

> Finished chain.
{'mission/text': '\n'
                 '\n'
                 'Our mission is to create comfortable and vibrant socks that '
                 'bring joy and help people express themselves through '
                 'fashion. We strive to provide quality socks that stand the '
                 'test of time and offer a wide variety of colors and designs '
                 'to fit any style.',
 'name/text': '\n\nSockarooni.',
 'product': 'colorful socks',
 'slogan/text': '\n\n"Step Into Color With Our Socks!"',
 'values/text': '\n'
                '\n'
                '1. Quality: Produce high quality socks that are designed to '
                'last.\n'
                '2. Creativity: Design innovative socks that are enjoyable to '
         

Setting the `concurrent` flag to `False` would run the child chains serially.

In [4]:
parallel_chain.concurrent=False
s = time.perf_counter()
output = parallel_chain("colorful socks")
pprint.pprint(output)
print('\033[1m' + f"Serial executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name" started.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


Child chain for key="name" finished after 23.66 seconds.
Child chain for key="mission" started.
Child chain for key="mission" finished after 3.05 seconds.
Child chain for key="slogan" started.
Child chain for key="slogan" finished after 1.34 seconds.
Child chain for key="values" started.
Child chain for key="values" finished after 6.15 seconds.

> Finished chain.
{'mission/text': '\n'
                 '\n'
                 'Our mission at [Company Name] is to bring color and joy to '
                 'the world by creating a vibrant and comfortable collection '
                 'of quality socks that make a bold statement and bring a '
                 'smile to everyone who wears them.',
 'name/text': '\n\nFancy Feet Socks.',
 'product': 'colorful socks',
 'slogan/text': '\n\n"Step Out in Style with Our Colorful Socks!"',
 'values/text': '\n'
                '\n'
                '1. Quality – produce high quality and durable colorful '
                'socks.\n'
                '2. In

## Nesting `ParallelChain`s
It is possible to nest `ParallelChain`s inside one another. Continuing from the previous example, we nest a concurrent `ParallelChain` inside the previous serial `ParallelChain`.

In [5]:
prompt_5_1 = PromptTemplate(
    input_variables=input_variables,
    template="Which gift would go well with {product}?",
)
chain_5_1 = LLMChain(llm=llm, prompt=prompt_5_1)

prompt_5_2 = PromptTemplate(
    input_variables=input_variables,
    template="What gift would not go well with {product}?",
)
chain_5_2 = LLMChain(llm=llm, prompt=prompt_5_2)

chain_5 = ParallelChain(
    input_variables=input_variables,
    chains={'good_gift': chain_5_1, 'bad_gift': chain_5_2},
    verbose=True,
    concurrent=True
)

parallel_chain.chains.update({'gift': chain_5})

output = parallel_chain("colorful socks")
pprint.pprint(output)



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="name" finished after 0.89 seconds.
Child chain for key="mission" started.
Child chain for key="mission" finished after 3.07 seconds.
Child chain for key="slogan" started.
Child chain for key="slogan" finished after 1.12 seconds.
Child chain for key="values" started.
Child chain for key="values" finished after 6.59 seconds.
Child chain for key="gift" started.


> Entering new ParallelChain chain...
Child chain for key="good_gift" started.
Child chain for key="bad_gift" started.
Child chain for key="bad_gift" finished after 0.79 seconds.
Child chain for key="good_gift" finished after 1.37 seconds.

> Finished chain.
Child chain for key="gift" finished after 1.37 seconds.

> Finished chain.
{'gift/bad_gift/text': '\n\nA book of crossword puzzles.',
 'gift/good_gift/text': '\n'
                        '\n'
                        'A colorful tie, a colorful scarf, a colorful hat, or '
         

We can now make the outer `ParallelChain` execute concurrently again by setting `parallel_chain.concurrent=True`. Executing this nested chain will result in executing a concurrent `ParallelChain` inside another concurrent `ParallelChain`.

In [6]:
parallel_chain.concurrent=True
output = parallel_chain("colorful socks")
pprint.pprint(output)



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="mission" started.
Child chain for key="slogan" started.
Child chain for key="values" started.
Child chain for key="gift" started.


> Entering new ParallelChain chain...
Child chain for key="good_gift" started.
Child chain for key="bad_gift" started.
Child chain for key="name" finished after 0.75 seconds.
Child chain for key="slogan" finished after 1.10 seconds.
Child chain for key="good_gift" finished after 1.17 seconds.
Child chain for key="bad_gift" finished after 1.36 seconds.

> Finished chain.
Child chain for key="gift" finished after 1.37 seconds.
Child chain for key="mission" finished after 3.51 seconds.
Child chain for key="values" finished after 6.58 seconds.

> Finished chain.
{'gift/bad_gift/text': '\n'
                       '\n'
                       'A gift card, as it does not have any physical material '
                       'and is not related to clothing.',
 'gift/good